# L9: Evaluation Part II

Évaluer les réponses des LLM lorsque qu'il n'y a pas de « bonne réponse » unique.

## Configuration
#### Chargement de la clé OpenAI et des librairies Python

In [ ]:
import os
from openai import OpenAI
import sys
sys.path.append('../..')
import utils
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('OPENAI_API_KEY')
)

In [ ]:
def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

### Parcourir le système de bout en bout pour répondre à la demande de l'utilisateur

Ces fonctions d'aide exécutent la chaîne de prompts que vous avez vue dans les vidéos précédentes.

In [ ]:
customer_msg = f"""
Parlez-moi du SmartX ProPhone et de l'appareil photo FotoSnap, le reflex numérique.
Quelles télévisions ou produits liés à la télévision avez-vous également ?"""

products_by_category = utils.get_products_from_query(customer_msg)
products_by_category = '\n'.join([line for line in products_by_category.split('\n') if not line.strip().startswith('```')])
category_and_product_list = utils.read_string_to_list(products_by_category)
product_info = utils.get_mentioned_product_info(category_and_product_list)
assistant_answer = utils.answer_user_msg(user_msg=customer_msg,
                                                   product_info=product_info)

In [ ]:
print(assistant_answer) 

### Évaluer la réponse du LLM à l'utilisateur selon un barème, basé sur les informations sur les produits extraites.

In [ ]:
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}

In [ ]:
def eval_with_rubric(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    system_message = """
    Vous êtes un assistant qui évalue la qualité des réponses du service client à 
    une question de l'utilisateur en examinant le contexte que l'agent du service 
    client utilise pour générer sa réponse. 
    """

    user_message = f"""
    Vous évaluez une réponse soumise à une question en fonction du contexte que l'agent 
    utilise pour répondre à la question. Voici les données :
    [DÉBUT DES DONNÉES]
    ************
    [Question] : {cust_msg}
    ************
    [Contexte] : {context}
    ************
    [Soumission] : {completion}
    ************
    [FIN DES DONNÉES]

    Comparez le contenu factuel de la réponse soumise avec le contexte.
    Ignorez toute différence de style, de grammaire ou de ponctuation.
    Répondez aux questions suivantes :

    La réponse de l'assistant est-elle basée uniquement sur le contexte fourni ? (O ou N)
    La réponse inclut-elle des informations qui ne sont pas fournies 
    dans le contexte ? (O ou N)
    Y a-t-il un désaccord entre la réponse et le contexte ? (O ou N)
    Comptez combien de questions l'utilisateur a posées. (output un nombre)
    Pour chaque question posée par l'utilisateur, y a-t-il une réponse correspondante ?
    Question 1 : (O ou N)
    Question 2 : (O ou N)
    ...
    Question N : (O ou N)
    Parmi le nombre de questions posées, combien d'entre elles ont été 
    abordées par la réponse ? (output un nombre) 
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

### Évaluez la réponse de l'LLM à l'utilisateur en fonction d'une réponse "idéale" / "expert" (générée par un humain).

In [ ]:
test_set_ideal = {
    'customer_msg': """
Parle-moi du SmartX Pro Phone et de la caméra FotoSnap, celle qui est DSLR.
Aussi, quels téléviseurs ou produits liés aux téléviseurs avez-vous ?""",
    'ideal_answer':"""
Bien sûr ! Le SmartX ProPhone est un smartphone puissant avec des fonctionnalités 
avancées pour la caméra. Par exemple, il possède un double appareil photo de 12 MP.
D'autres caractéristiques incluent la connectivité 5G et 128 Go de stockage.
Il a également un écran de 6,1 pouces. Le prix est de 899,99 $.

La caméra reflex numérique FotoSnap est idéale pour capturer des photos et des vidéos 
époustouflantes. Parmi ses fonctionnalités, on trouve la vidéo en 1080p, un écran LCD 
de 3 pouces, un capteur de 24,2 MP et des objectifs interchangeables.
Le prix est de 599,99 $.

Pour les téléviseurs et les produits connexes, nous proposons 3 téléviseurs.

Tous les téléviseurs offrent la HDR et les fonctionnalités Smart TV.

Le téléviseur CineView 4K présente des couleurs vives et des fonctionnalités intelligentes.
Parmi ces fonctionnalités, on trouve un écran de 55 pouces et une résolution 4K.
Il est au prix de 599 $.

Le téléviseur CineView 8K est un téléviseur 8K époustouflant.
Parmi ses fonctionnalités, on trouve un écran de 65 pouces et une résolution 8K.
Il est au prix de 2999,99 $.

Le téléviseur CineView OLED vous permet de vivre des couleurs vives.
Parmi ses fonctionnalités, on trouve un écran de 55 pouces et une résolution 4K.
Il est au prix de 1499,99 $.

Nous proposons également 2 produits de home cinéma, tous deux dotés de Bluetooth.
Le home cinéma SoundMax est un système puissant pour une expérience audio immersive.
Ses caractéristiques incluent un son surround 5.1, une puissance de 1000W et un
caisson de basses sans fil. Il est au prix de 399,99 $.

La barre de son SoundMax est une barre de son élégante et puissante.
Ses caractéristiques incluent un son surround 2.1, une puissance de 300W et un
caisson de basses sans fil. Elle est au prix de 199,99 $.

Avez-vous d'autres questions supplémentaires concernant ces produits que vous
avez mentionnés ici ? Ou avez-vous d'autres questions auxquelles je peux vous aider ?
    """
}

### Vérifiez si la réponse de l'IA est d'accord ou en désaccord avec la réponse de l'expert.

Le prompt provient du projet [OpenAI evals](https://github.com/openai/evals/blob/main/evals/registry/modelgraded/fact.yaml).

[BLEU score](https://en.wikipedia.org/wiki/BLEU): Une autre façon d'évaluer si deux morceaux de texte sont similaires ou non.

In [ ]:
def eval_vs_ideal(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    Vous êtes un assistant qui évalue la qualité des réponses d'un agent du 
    service client à une question d'utilisateur en comparant la réponse à la 
    réponse idéale (d'expert). Output a single letter and nothing else.
    """

    user_message = f"""\
    Vous comparez une réponse soumise à une réponse d'expert sur une question donnée. Voici les données :
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Expert]: {ideal}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Comparez le contenu factuel de la réponse soumise avec la réponse de l'expert. Ignorez les différences de style, de grammaire ou de ponctuation.
La réponse soumise peut être soit un sous-ensemble, soit un sur-ensemble de la réponse de l'expert, ou elle peut être en conflit avec celle-ci. Déterminez quel cas s'applique. Répondez à la question en sélectionnant l'une des options suivantes :
(A) La réponse soumise est un sous-ensemble de la réponse de l'expert et est entièrement cohérente avec elle.
(B) La réponse soumise est un sur-ensemble de la réponse de l'expert et est entièrement cohérente avec elle.
(C) La réponse soumise contient tous les mêmes détails que la réponse de l'expert.
(D) Il y a un désaccord entre la réponse soumise et la réponse de l'expert.
(E) Les réponses diffèrent, mais ces différences n'ont pas d'importance du point de vue de la factualité.
choice_strings : ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [ ]:
print(assistant_answer)

In [ ]:
eval_vs_ideal(test_set_ideal, assistant_answer)

In [ ]:
assistant_answer_2 = "La vie est comme une boite de chocolats"

In [ ]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)